## 141 期間内におけるmonth, weekのカバー率

In [5]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Data Load

In [2]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = utils.read_df_pkl('../input/train0*')
df_test = utils.read_df_pkl('../input/test0*')

df_train = reduce_mem_usage(df_train)
df_test  = reduce_mem_usage(df_test )
train_test = pd.concat([df_train, df_test], axis=0)
train_test   = reduce_mem_usage(train_test  )

100%|██████████| 3/3 [00:00<00:00, 82.05it/s]


Mem. usage decreased to  5.58 Mb (48.2% reduction)
Mem. usage decreased to  3.18 Mb (43.8% reduction)
Mem. usage decreased to  9.00 Mb (0.0% reduction)


In [16]:
train_test['first_active_month'] =  train_test['first_active_month'].astype('str').map(lambda x:
                                     '2012' if x <='2012-12'
#                                      else '2012-2' if x <='2012-06'
#                                      else '2012-3' if x <='2012-09'
#                                      else '2012-4' if x <='2012-12'
#                                      else '2013-1' if x <='2013-03'
#                                      else '2013-2' if x <='2013-06'
#                                      else '2013-3' if x <='2013-09'
                                     else '2013' if x <='2013-12'
#                                      else '2014-1' if x <='2014-03'
#                                      else '2014-2' if x <='2014-06'
#                                      else '2014-3' if x <='2014-09'
                                     else '2014' if x <='2014-12'
#                                      else '2015-1' if x <='2015-03'
#                                      else '2015-2' if x <='2015-06'
#                                      else '2015-3' if x <='2015-09'
                                     else '2015' if x <='2015-12'
                                     else '2016-1' if x <='2016-03'
                                     else '2016-2' if x <='2016-06'
                                     else '2016-3' if x <='2016-09'
                                     else '2016-4' if x <='2016-12'
                                     else '2017-1' if x <='2017-03'
                                     else '2017-2' if x <='2017-06'
                                     else '2017-3' if x <='2017-09'
                                     else '2017-4' if x <='2017-12'
                                     else '2018-1' if x <='2018-03'
                                     else '2018-2' if x <='2018-06'
                                     else '2018-3' if x <='2018-09'
                                     else '2018-4' if x <='2018-12'
                                                                         else '2019'
                                    )

In [18]:
from preprocessing import get_dummies
cat_list = [col for col in train_test.columns if col.count('feature') or col.count('month')]
df_ohe = get_dummies(df=train_test, cat_list=cat_list )
df_ohe.head()

,card_id,target,dummie_feature_1_1,dummie_feature_1_2,dummie_feature_1_3,dummie_feature_1_4,dummie_feature_1_5,dummie_feature_2_1,dummie_feature_2_2,dummie_feature_2_3,dummie_feature_3_0,dummie_feature_3_1,dummie_first_active_month_2012,dummie_first_active_month_2013,dummie_first_active_month_2014,dummie_first_active_month_2015,dummie_first_active_month_2016-1,dummie_first_active_month_2016-4,dummie_first_active_month_2017-1,dummie_first_active_month_2017-4,dummie_first_active_month_2018-1,dummie_first_active_month_2018-4,dummie_first_active_month_2019
0,C_ID_92a2005557,-0.820312,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0
1,C_ID_3d0044924f,0.392822,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,C_ID_d639edf6cd,0.687988,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0
3,C_ID_186d6a6901,0.142456,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0
4,C_ID_cdbd2c0db2,-0.159790,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0


In [24]:
df_train = df_ohe[~df_ohe[target].isnull()]
df_test = df_ohe[df_ohe[target].isnull()]

### make ratio feature

In [27]:
fname = '156_ffm'
ignore_features = ['first_active_month', 'card_id', target]
# df_train = base_train
# df_test = base_test

for col in df_train.columns:
    if col in ignore_features: continue
#     if not(col.count('ratio')):continue
    utils.to_pkl_gzip(path = f'../features/ffm_winner/{fname}_train_{col}@', obj=df_train[col].values)
    utils.to_pkl_gzip(path = f'../features/ffm_winner/{fname}_test_{col}@', obj=df_test[col].values)